In [1]:
import fastText
import pandas as pd
from pathlib import Path

import json
from cleantext import clean
import random

import sklearn.metrics

In [2]:
all_classes = ['claudience', 'clpersuasive', 'clagreement', 'cldisagreement', 'clinformative', 'clmean', 'clcontroversial', 'cltopic', 'clsentiment']

In [3]:
def each_row(row):
    text = clean(row['text'], lower=True, no_line_breaks=True, zero_digits=True, fix_unicode=True, to_ascii=True)
    text = f"__label__{int(row[cls])} {text}"
    return text

In [4]:
def to_file(df, cls, kind):
    df = df.dropna(subset=[cls])
    df['out'] = df.apply(each_row, axis=1)
    out_p = f"/mnt/data/group07/johannes/ynacc_proc/fasttext_baseline/cls/{cls}/"
    Path(out_p).mkdir(parents=True, exist_ok=True) 
    out_text = '\n'.join(df['out'].values.tolist())
    Path(out_p + kind).write_text(out_text)
    return str(Path(out_p + kind))

In [5]:
def run(ps, i):
    lr = random.uniform(0, 1)
    epoch=round(random.uniform(5, 50))
    wordNgrams=round(random.uniform(1, 5))
    minCount=round(random.uniform(1, 100))
    model = fastText.train_supervised(input=ps[0], lr=lr, epoch=epoch, wordNgrams=wordNgrams, minCount=minCount)

#     had to do it like this because I want to get the prediction and not just a metric from the model
    preds = Path(ps[1]).read_text().split('\n')
    
    truth = []
    output = []
    for p in preds:
        label = p[:10]
        text =  p[11:]
        truth.append(label)
        output.append(model.predict(text)[0][0])

    rpt = sklearn.metrics.classification_report(truth, output, output_dict=True)

    rpt['lr'] = lr
    rpt['epoch'] = epoch
    rpt['wordNgrams'] = wordNgrams
    rpt['minCount'] = minCount
    rpt['kappa'] = sklearn.metrics.cohen_kappa_score(truth, output)
    Path(ps[0].replace('train.csv', f"{i}_results.json")).write_text(json.dumps(rpt))

In [6]:
for cls in all_classes:
    ps = []
    for x in ['train.csv', 'val.csv']:
        df = pd.read_csv('/mnt/data/group07/johannes/ynacc_proc/replicate/split/' + x, usecols=['text', cls])
        ps.append(to_file(df, cls, x))
    for i in range(10):
        run(ps, i)

/home/group7/anaconda3/envs/fasttext/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: 